<a href="https://colab.research.google.com/github/siddharthsinghchaudhari/python/blob/main/Eye_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras.layers import Input, Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os
from numpy import argmax

In [ ]:
path = "/content/drive/MyDrive/DATA/"

In [ ]:
import os
data_dir_list = os.listdir(path)
data_dir_list


['training', 'testing', 'diseases']

# New section

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
datagen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               vertical_flip=True,
                               fill_mode='nearest',# Fill in missing pixels with the nearest filled value
                              )

In [ ]:
img_rows=224
img_cols=224
num_channel=3

num_epoch = 100
batch_size = 32

img_data_list=[]
classes_names_list=[]
target_column=[]
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print(dataset)
    print("Getting image from {} folder".format(dataset))
    img_list= os.listdir(path +"/"+ dataset)
    for img in img_list:
        input_img = cv2.imread(path +"/"+dataset+"/"+img)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

training
Getting image from training folder


error: ignored

In [ ]:
|num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list) # convert images in numpy array
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape
print("number of samples",num_of_samples)
print("target column before encoding",target_column)

In [ ]:
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
np.unique(target_column)

In [ ]:
target_column

In [ ]:
target_column_hotcoded = to_categorical(target_column,num_classes)
X,Y = shuffle(img_data,target_column_hotcoded,random_state=2)
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2)

In [ ]:
CNN_MODLE = Sequential()

CNN_MODLE.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
CNN_MODLE.add(Conv2D(64,(3,3),activation='relu'))
CNN_MODLE.add(MaxPool2D(pool_size=(2,2)))
CNN_MODLE.add(Dropout(0.2))

CNN_MODLE.add(Conv2D(128,(3,3),activation='relu'))
CNN_MODLE.add(Conv2D(128,(3,3),activation='relu'))
CNN_MODLE.add(MaxPool2D(pool_size=(2,2)))
CNN_MODLE.add(Dropout(0.2))

CNN_MODLE.add(Flatten())
CNN_MODLE.add(Dense(128,activation='relu'))
CNN_MODLE.add(Dropout(0.2))
CNN_MODLE.add(Dense(num_classes,activation='softmax'))
CNN_MODLE.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

CNN_MODLE.summary()

In [ ]:
hist = CNN_MODLE.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,validation_data=(X_test,y_test))


In [ ]:
print(hist.history.keys())
loss_train = hist.history['loss']
loss_val = hist.history['val_loss']
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = hist.history['accuracy']
loss_val = hist.history['val_accuracy']
print(len(loss_train), len(loss_val))
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
cnn_score = CNN_MODLE.evaluate(X_test,y_test,batch_size=batch_size)
y_predicted_cnn = CNN_MODLE.predict(X_test)
print('Test Loss',cnn_score[0])
print("Test Accuracy",cnn_score[1])

In [ ]:
y_pred=np.argmax(y_predicted_cnn, axis=1)
y_test_cnn=np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_cnn, y_pred))

In [ ]:
cm = confusion_matrix(y_test_cnn, y_pred)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
CNN_MODLE.save_weights('CNN.h5')

In [ ]:
from keras.applications.vgg19 import VGG19
vgg19_model = VGG19(weights = "imagenet", include_top=False, input_shape = (224,224,3))

In [ ]:
vgg19_model.summary()

In [ ]:
for layer in vgg19_model.layers:
    layer.trainable = False

#Adding custom Layers
x = vgg19_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(4, activation="softmax")(x)

In [ ]:
vgg19_model = Model(inputs=vgg19_model.input, outputs=predictions)

In [ ]:
vgg19_model.summary()

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=0.0001)
vgg19_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = vgg19_model.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,validation_data=(X_test,y_test))


In [ ]:
vgg19_model.save_weights('VGG19.h5')

In [ ]:
print(hist.history.keys())
loss_train = hist.history['loss']
loss_val = hist.history['val_loss']
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
loss_train = hist.history['accuracy']
loss_val = hist.history['val_accuracy']
print(len(loss_train), len(loss_val))
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
vgg_score = vgg19_model.evaluate(X_test,y_test,batch_size=batch_size)
y_predicted_vgg = vgg19_model.predict(X_test)
print('Test Loss',vgg_score[0])
print("Test Accuracy",vgg_score[1])

In [ ]:
y_pred=np.argmax(y_predicted_vgg, axis=1)
y_test_vgg=np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_vgg, y_pred))

In [ ]:
cm = confusion_matrix(y_test_vgg, y_pred)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
plotdata = pd.DataFrame({

    "cnn": cnn_score,
    "vgg19":vgg_score},
    index=["Loss", "Accuracy"])
plotdata.plot(kind="bar",figsize=(10, 10))
plt.title("Model comparision")
plt.xlabel("Index")
plt.ylabel("Percentage")
